In [1]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules import loss
import torch.optim as optim

In [3]:
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, tagset_size, batch_size):
        # 親クラスのコンストラクタ。決まり文句
        super(LSTMClassifier, self).__init__()
        # 隠れ層の次元数。これは好きな値に設定しても行列計算の過程で出力には出てこないので。    
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        # LSTMの隠れ層。これ１つでOK。超便利。
        self.lstm = nn.LSTM(embedding_dim, hidden_dim//2, batch_first=True, bidirectional=True )
        # LSTMの出力を受け取って全結合してsoftmaxに食わせるための１層のネットワーク
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        # softmaxのLog版。dim=0で列、dim=1で行方向を確率変換。
        # self.softmax = 
    
    def forward(self, x):
        #embeds.size() = (batch_size × len(sentence) × embedding_dim)
        batch_size, seq_len = x.shape[0], x.shape[1]
        _, hidden_layer = self.lstm(x)
        # print(hidden_layer)
        bilstm_out = torch.cat([hidden_layer[0][0], hidden_layer[0][1]], dim=1)
        # y = self.hidden2tag(hidden_layer[0].view(batch_size, -1))

        y = self.hidden2tag(bilstm_out)
        y = F.log_softmax(y, dim=1)
        return y

In [4]:
import pickle
class DataManager:
    def __init__(self, data_path) -> None:
        import os
        import pickle
        self.data_path = data_path
        os.makedirs(data_path, exist_ok=True)
        self.dir = os.listdir(data_path)

    def is_exist(self, name):
        return (name in self.dir)
    
    def save_data(self, name, obj):
        with open(self.data_path+name, "wb") as f:
            pickle.dump(obj, f)
        print("success save : {0}{1}".format(self.data_path, name))

    def load_data(self, name):
        with open(self.data_path+name, "rb") as f:
            obj = pickle.load(f)
        print("success load : {0}{1}".format(self.data_path, name))
        return obj

In [5]:
from pyknp import Juman
from sentence_transformers import SentenceTransformer
import scipy.spatial
Nmodel_path = "/home/yamada/Downloads/training_bert_japanese"
Nmodel = SentenceTransformer(Nmodel_path, show_progress_bar=False)
emb_dim = Nmodel.encode(["お辞儀をしている男性会社員"])[0].shape[0]

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [6]:
def make_X(convs, max_len):
    # emb_dim = nlp("形態素").vector.shape
    X_data = []
    
    for conv in convs :
        # vec_list = np.zeros( (max_len, emb_dim[0]) )
        sentence_vectors = Nmodel.encode(conv)
        # for i, ut in enumerate(conv):
        #     doc = nlp(ut)
        #     vec_list[i] = doc.vector
        X_data.append(sentence_vectors)
    return np.array(X_data)

In [7]:
path = "../hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']

output = "./"

In [8]:
def read_json_with_NoErr(path:str, datalist:list) -> pd.DataFrame:
    cols = ['did', 'tid', 'usr', 'sys', 'ec']
    df = pd.DataFrame(index=[], columns=cols)

    for p in datalist:
        datapath = Path(path + p + '/')
        for file in datapath.glob("*.json"):
            with open(file, "r") as f:
                json_data = json.load(f)
                did = json_data["did"]
                for t in json_data["turns"]:
                    if t["turn-index"] == 0:
                        continue
                    if t["speaker"] == "U":
                        usr = t["utterance"]
                        continue
                    if t["speaker"] == "S" :
                        tid = t["turn-index"]
                        sys = t["utterance"]
                        if t["error_category"]:
                            ec = t["error_category"]
                        else:
                            ec = ["No-Err"]
                        df = df.append(pd.DataFrame([did, tid, usr, sys, ec], index = cols).T)
    df.reset_index(inplace=True, drop=True)
    return df

In [9]:
df = read_json_with_NoErr(path, datalist)

In [10]:
def extract_continue_convs_with_error(df, length, errors):
    new_convs = []
    continue_conv = []
    did = 0
    for d, u, s, e in zip(df.did, df.usr, df.sys, df.ec):
        # did が変化すれば，別の対話
        if d != did:
            continue_conv = []
            did = d
        continue_conv .append(u)
        continue_conv .append(s)
        for err in errors:
            if len(continue_conv) >= length and err in e:
                new_convs.append( continue_conv[-length:] )
    
    return new_convs

In [11]:
errors = ["Topic transition error", "Unclear intention", "Lack of information"]
# errors = ["Lack of information"]
errors = ["Topic transition error"]
# errors = ["Unclear intention"]
length = 3

In [12]:
UT = extract_continue_convs_with_error(df, length, errors)

In [13]:
import random
UT_NO_ALL = extract_continue_convs_with_error(df, length, [ "No-Err"])
UT_NO = random.sample(UT_NO_ALL, len(UT))
# UT_NO = UT_NO_ALL

In [14]:
print(len(UT))
print(len(UT_NO))

174
174


In [15]:
UT

[['スイカは実がなってたことあります', '意味が分かりません', 'あなたの好きな映画について教えてください'],
 ['焼きそばは好きですか。麺が美味しいですね', 'そうですね。スポーツは好きですか？', '麺が美味しいですね'],
 ['麺類は食べやすいですね', '旅行は好きですか？', 'パスタが好きですね'],
 ['ケースバイケースで判断するんですね',
  'はい、長いこと彼女いないんでいろいろ考えちゃいます。',
  'そういえば、よく旅行に行くと言っていましたが、ケースバイケースで臨機応変に対応するのでしょうか？'],
 ['紅玉を求めるかもしれない', 'リンゴの紅玉が好きなんですね。', 'そういえば、この間、旅行したと伺いましたが、奇遇に驚きます'],
 ['いいですね！', '寒いのは苦手ですか？', 'そうでしたか！\u3000えっと、好きなスポーツは何ですか？'],
 ['行くまでが楽しいですね', 'その気持ち凄く分かる。', '今日も買い物には行かれたんですか？'],
 ['高波がすごいですよねえ', '泳ぐのは好きですか？', '読書だそうですが、最近は本を読んでおられますか？'],
 ['天気がいいとそれだけで元気が出ちゃうよね。',
  'こんにちは。今日はいい天気ですね。',
  'あ\u3000そうなんですか。\u3000うーんとね、猫を飼ったことはありますか？'],
 ['インターネットで調べたところ、もしかすると炊き込みご飯です。',
  'お昼から炊き込みご飯を作るなんてえらいですね。',
  '日本では１９８４年に大学同士で接続されたのが始まりなんだ。'],
 ['日本では１９８４年に大学同士で接続されたのが始まりなんだ。',
  'インターネットはいまではだれでも使っていますよね。',
  '日本の国花は桜だけど、法律で決められたものじゃないんだよ。'],
 ['元気ですよ',
  'それは良かった。夏バテしてない？',
  'ところで、気になったニュースがあるんですが、京都大霊長類研究所の研究チームが、生まれてまもない赤ちゃんがほほ笑む「自発的微笑」と呼ばれる動作が、ニホンザルの赤ちゃんにもあると確認したそうですよ。'],
 ['モカ、マウンテン、キリマンジャロ……自分のお気に入りのコーヒーがあると、カッ

In [16]:
model_path = "../models/context/"
model_name = "topic3-{0}.pickle".format(length)
modelM = DataManager(model_path)
print(model_name)
if modelM.is_exist(model_name):
    model = modelM.load_data(model_name)

topic3-3.pickle
success load : ../models/context/topic3-3.pickle


In [17]:
X = make_X(UT + UT_NO, length)
y = np.concatenate( [ np.ones(len(UT)), np.zeros(len(UT_NO)) ] )

In [18]:
with torch.no_grad():
    X_tensor = torch.tensor(X, device='cuda:0').float()
    y_tensor = torch.tensor(y, dtype=torch.long, device='cuda:0')
            # 推論
    y_pred= np.array(model(X_tensor).cpu()).argmax(axis=1)

/home/yamada/.local/lib/python3.6/site-packages/torch/nn/modules/rnn.py:662: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  /pytorch/aten/src/ATen/native/cudnn/RNN.cpp:915.)
  self.dropout, self.training, self.bidirectional, self.batch_first)


In [19]:
metrics.accuracy_score(y, y_pred)

0.5632183908045977

In [20]:
y

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [21]:
y_pred

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1,

In [22]:
UT_NO

[['塩加減がちょうどいいものがいいですね', '塩辛なんかも塩加減が色々あって、好みの味は甘口ですよ。', '焼肉もしながらとかできるのかな？'],
 ['挑戦したんですか！！すげ〜！！じゃあ今年は一個いけるようにがんばってください!',
  '最近はもう長距離泳げなくなりましたが頑張ります。あなたはスポーツは何かしていますか？',
  'え、マラソンかな'],
 ['大阪の世界最大級の水族館である海遊館の飼育員さんはサメの水槽の上から一気に餌を流し込むなんて、とても豪快ですね。',
  '迫力があって楽しそうですね！',
  '海遊館にはイルカはいませんが、大水槽で泳ぐジンベイザメが有名で、しかし残念ながら海遊館のジンベイザメは最近亡くなってしまったそうです。'],
 ['気にせず寝てますよ。', 'あなたも寝ることはあるんですか？', 'そうします'],
 ['読書は楽しいですね', 'そうですね。', 'そうですよ'],
 ['そうですね、秋晴れですね。', '空気が澄んでいます', '木陰で涼しいですしね。'],
 ['アニメもおもしろいですよね。私からの質問になってしまいますが、最近楽しみにしてる事は何ですか？',
  'もうすぐ旅行にいくのでそれが楽しみです',
  'おっ、私もなんですよ。場所はどこですか？'],
 ['うちのチビは冷たいですね。名前呼んでも無視するくせに、エサの時は近づいてきますから。',
  'まあネコなんて、そんなもんですよね。',
  'イタリア、フランス、イギリス、香港くらいです。'],
 ['どんな本を読みますか？', 'ミステリーですね', '面白いですよね。よく小説とか読まれたりします？'],
 ['はい、でもまだ興味にとどまっています。', '旅行はすきですか？', '越前海岸の温泉とか'],
 ['旅行を振り返りますか', '夏の北海道が良かったです', '北海道は懐かしいですね'],
 ['ずばり鍋でしょう！', '鍋は美味しいですよね', 'ズバリ鍋！！私も鍋大好きです。先日も友人ともつ鍋を食べに行きました。'],
 ['スロットがもっとも盛り上がっていた時代ですね。', 'そうですね', 'では、好きな食べ物は何ですか？'],
 ['明日は晴れるは2003年のアルバムですね', 'そうなんですか', 'そーな